In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ptflops import get_model_complexity_info
from einops import rearrange, repeat

from CrossAttn import CrossAttn
from deepSense import DeepSense
from GlobalFusion import GlobalFusion
from ALAE import ALAE_TAE
from Attend import AttendDiscriminate
from deepconvlstm import DeepConvLSTM
from ifconv import If_ConvTransformer_W
from TinyHAR import TinyHAR_Model
from MixerMLP import FFTMIXER_HAR_Model
from mcnn import MCNN
from deepconvlstm_attn import DeepConvLSTM_ATTN
from visionmixer import Vision_MIXER

In [2]:
window_length = 128
sensor_channel = 9
filter_num = 32
nb_classes = 12
model  = TinyHAR_Model((1,1, window_length, sensor_channel ), 
                       nb_classes,
                       filter_num = filter_num,#config["filter_num"],
                       cross_channel_interaction_type = "attn",    # attn  transformer  identity
                       cross_channel_aggregation_type = "FC",  # filter  naive  FC
                       temporal_info_interaction_type = "lstm",     # gru  lstm  attn  transformer  identity
                       temporal_info_aggregation_type = "tnaive")    # naive  filter  FC )
y = model(torch.randn((1,1, window_length, sensor_channel )))
print(y)

tensor([[-0.1056,  0.0702,  0.1043, -0.1146, -0.0097, -0.0070,  0.0428, -0.0394,
          0.0921, -0.0025, -0.0145, -0.0359]], grad_fn=<AddmmBackward>)


In [3]:
data_dict = [
    {"dataset_name":"pamap2",  "sensor_channel":18, "window_length":128, #
     "fft_window_length":8, "nb_classes":18, "fft_feature_dim":32, #
     "S_number_sensors_type":2,"L_sensor_locations":3},#
    {"dataset_name":"dsads",  "sensor_channel":45, "window_length":125, #
     "fft_window_length":5, "nb_classes":19, "fft_feature_dim":50, #
     "S_number_sensors_type":3,"L_sensor_locations":5},#
    {"dataset_name":"dg",  "sensor_channel":9, "window_length":64, #
     "fft_window_length":4, "nb_classes":2, "fft_feature_dim":32, #
     "S_number_sensors_type":1,"L_sensor_locations":3},#
    # # {"dataset_name":"rw",  "sensor_channel":45, "window_length":100, #
    # #  "fft_window_length":10, "nb_classes":8, "fft_feature_dim":20, #
    # #  "S_number_sensors_type":3,"L_sensor_locations":5},#
    {"dataset_name":"hapt",  "sensor_channel":6, "window_length":128, #
     "fft_window_length":8, "nb_classes":12, "fft_feature_dim":32, #
     "S_number_sensors_type":2,"L_sensor_locations":1},#
    {"dataset_name":"motionsense",  "sensor_channel":12, "window_length":128, #
     "fft_window_length":8, "nb_classes":6, "fft_feature_dim":32, 
     "S_number_sensors_type":4,"L_sensor_locations":1},#
    # {"dataset_name":"skoda",  "sensor_channel":30, "window_length":128, #
    #  "fft_window_length":8, "nb_classes":10, "fft_feature_dim":32, #
    #  "S_number_sensors_type":1,"L_sensor_locations":10},#
    # {"dataset_name":"GesHome",  "sensor_channel":9, "window_length":50,# 
    #  "fft_window_length":7, "nb_classes":18, "fft_feature_dim":20, #
    #  "S_number_sensors_type":3,"L_sensor_locations":1},#
    # {"dataset_name":"lsign",  "sensor_channel":15, "window_length":75, #
    #  "fft_window_length":6, "nb_classes":36, "fft_feature_dim":40, #
    #  "S_number_sensors_type":1,"L_sensor_locations":5},#
    # {"dataset_name":"emgg",  "sensor_channel":9, "window_length":200, #
    #  "fft_window_length":10, "nb_classes":7, "fft_feature_dim":40, #
    #  "S_number_sensors_type":1,"L_sensor_locations":3},#
    # {"dataset_name":"hhar",  "sensor_channel":6, "window_length":200, #
    #  "fft_window_length":10, "nb_classes":6, "fft_feature_dim":40, #
    #  "S_number_sensors_type":2,"L_sensor_locations":1},#
    {"dataset_name":"mhealth",  "sensor_channel":18, "window_length":128, #
     "fft_window_length":8 , "nb_classes":12, "fft_feature_dim":32, #
     "S_number_sensors_type":3,"L_sensor_locations":2}#
]


In [4]:
# mcnn globalfusion deepsense bianda
# dcl bian 
# qiyu baochi bubian

In [5]:
results={}
for data in data_dict:
    #print(data)
    result_data = {}
    dataset_name=data["dataset_name"]
    sensor_channel=data["sensor_channel"]
    window_length=data["window_length"]
    fft_window_length=data["fft_window_length"]
    nb_classes=data["nb_classes"]
    fft_feature_dim=data["fft_feature_dim"]
    S_number_sensors_type=data["S_number_sensors_type"]
    L_sensor_locations=data["L_sensor_locations"]
    
    #  ----------------  build MCNN model ---------------------


    config = { "nb_conv_blocks": 2,
              "nb_filters": 64,
              "dilation": 1,
              "batch_norm": 0,
              "filter_width": 5,
              "drop_prob": 0.25}
    model  = MCNN((1,1, window_length, sensor_channel ), 
                   nb_classes,
                  1,
                  config)


    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)

    result_data["mcnn_macs"] = macs
    result_data["mcnn_params"] = params
    
    #  ----------------  build DCL model ---------------------

    config = {  
        "nb_conv_blocks": 2,
      "nb_filters": 64,
      "dilation": 1,
      "batch_norm": 0,
      "filter_width": 7,
      "nb_layers_lstm": 1,
      "drop_prob": 0.2,
      "nb_units_lstm": 128}



    input_shape = (1, 1,window_length,sensor_channel)
    model = DeepConvLSTM(input_shape, nb_classes, 1,config=config)
    #x = torch.rand(1, 1, window_length, sensor_channel)
    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)

    result_data["dcl_macs"] = macs
    result_data["dcl_params"] = params

    # -----------------build DeepConvLSTM Attend -------------





    config = {  
        "nb_conv_blocks": 2,
        "nb_filters": 64,
        "dilation": 1,
        "batch_norm": 0,
        "filter_width": 5,
        "nb_layers_lstm": 2,
        "drop_prob": 0.5,
        "nb_units_lstm": 128}
    model  = DeepConvLSTM_ATTN((1,1, window_length, sensor_channel ), 
                               nb_classes,
                               1,
                               config)

    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)

    result_data["dcl_attn_macs"] = macs
    result_data["dcl_attn_params"] = params
    # -------------- build DeepSense Model 

    if fft_feature_dim<=21:
        kernel_size = 3
    else:
        kernel_size = 5
    model = DeepSense(input_shape=(1, 1, window_length, sensor_channel),   fft_segments_length = int(fft_feature_dim/2),   
                      k_number_sensors_group=S_number_sensors_type*L_sensor_locations, nb_classes= nb_classes, kernel_size_1=kernel_size)

    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)
    result_data["DeepSense_macs"] = macs
    result_data["DeepSense_params"] = params

    # ==============build  Globalfusion model 

    model = GlobalFusion(input_shape=(1, 1, window_length, sensor_channel),      fft_segments_length = int(fft_feature_dim/2),   
                         S_number_sensors_type=S_number_sensors_type, 
                         L_sensor_locations= L_sensor_locations, nb_classes= nb_classes, kernel_size_1 = kernel_size)


    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=True)

    result_data["GlobalFusion_macs"] = macs
    result_data["GlobalFusion_params"] = params

    # ---------------- Build Attend
    input_shape = (1, 1,window_length,sensor_channel)
    config = {"hidden_dim": 128,
      "filter_num": 64,
      "filter_size": 5,
      "enc_num_layers": 2,
      "dropout": 0.5,
      "dropout_rnn": 0.25,
      "dropout_cls": 0.5,
      "activation": "ReLU",
      "sa_div": 1}
    model = AttendDiscriminate(input_shape, nb_classes, config=config)
    #x = torch.rand(1, 1, window_length, sensor_channel)
    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)


    result_data["Attend_macs"] = macs
    result_data["Attend_params"] = params
    # ---------- Build ALAE ----------


    model = ALAE_TAE(input_shape=(1, 1, window_length, sensor_channel),      
                     nb_classes=nb_classes)

    #x = torch.rand(1, 1, window_length, sensor_channel)
    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)


    result_data["ALAE_macs"] = macs
    result_data["ALAE_params"] = params

    
    # ---------Build  IF Conv --------------


    
    if sensor_channel%9==0:
        
        fusion = True
    else:
        fusion = False
    model = If_ConvTransformer_W(1, sensor_channel, 32, 5, 3, 2, 64, 1, 0.2, dataset_name, window_length, nb_classes, fusion = fusion)
    
    #input = torch.randn(1, 1,sensor_channel , window_length)

    
    
    macs, params = get_model_complexity_info(model, (1,sensor_channel , window_length), as_strings=True,
                                           print_per_layer_stat=False, verbose=False)


    result_data["IfConv_macs"] = macs
    result_data["IfConv_params"] = params


    # -------------------- Build TinyHAR ----------------

    filter_num = 32
    model  = TinyHAR_Model((1,1, window_length, sensor_channel ), 
                           nb_classes,
                           filter_num = filter_num,#config["filter_num"],
                           cross_channel_interaction_type = "attn",    # attn  transformer  identity
                           cross_channel_aggregation_type = "FC",  # filter  naive  FC
                           temporal_info_interaction_type = "lstm",     # gru  lstm  attn  transformer  identity
                           temporal_info_aggregation_type = "tnaive")    # naive  filter  FC )
    
    macs, params = get_model_complexity_info(
        model, 
        (1, window_length, sensor_channel), 
        as_strings=True, print_per_layer_stat=False, verbose=False)


    result_data["tinyhar_macs"] = macs
    result_data["tinyhar_params"] = params

    results[dataset_name] = result_data


    # -------------------- Build Visionmixer ----------------


    model  = Vision_MIXER(
                input_shape=(1,1, window_length, sensor_channel ), 
                patch_size = 3,
                number_class= nb_classes) 
    
    macs, params = get_model_complexity_info(
        model, 
        (1, window_length, sensor_channel), 
        as_strings=True, print_per_layer_stat=False, verbose=False)


    result_data["Vision_MIXER_macs"] = macs
    result_data["Vision_MIXER_params"] = params

    results[dataset_name] = result_data
    # --------------- Build cross Attention -------------


    # model = CrossAttn(
    #     Ts_input_shape = (1,1,window_length,sensor_channel),
    #     hidden_dim   = 24,
    #     FFT_input_shape = (1,fft_feature_dim,fft_window_length,sensor_channel)
    # )


    # def prepare_input(resolution):
    #     x1 = torch.FloatTensor(1, 1, window_length, sensor_channel)
    #     x2 = torch.FloatTensor(1, fft_feature_dim, fft_window_length, sensor_channel)
    #     return dict(x = [x1, x2])



    # macs, params = get_model_complexity_info(model, 
    #                                          (1, window_length, sensor_channel), 
    #                                          input_constructor=prepare_input, 
    #                                          as_strings=True, print_per_layer_stat=False, verbose=False)

    # result_data["CrossAttn_macs"] = macs
    # result_data["CrossAttn_params"] = params


    

    # results[dataset_name] = result_data

    # ----------------- build the Mixer Model -------------

    # config["fft_mixer_share_flag"] = False
    # config["fft_mixer_temporal_flag"]  = True
    # config["fft_mixer_FFT_flag"]  = True
    # model  = FFTMIXER_HAR_Model(input_shape=(1,1, window_length, sensor_channel ), 
    #                             number_class = nb_classes,
    #                             filter_num = 6,
    #                             fft_mixer_segments_length = int(fft_feature_dim/2),
    #                             expansion_factor = 0.3,
    #                             fft_mixer_layer_nr = 2,
    #                             fuse_early = False,
    #                             temporal_merge= True,
    #                             oration = 0.25,
    #                             model_config = config)
    
    # macs, params = get_model_complexity_info(model, (1,window_length , sensor_channel), as_strings=True,
    #                                        print_per_layer_stat=False, verbose=False)


    # result_data["Mixer_macs"] = macs
    # result_data["Mixer_params"] = params

In [ ]:
Number of Parameters : 387910 -------- Model Complexity : 105.31 MMac ------------ Number of parameters 2: 387.91 k 

In [6]:
results["mhealth"]

{'mcnn_macs': '58.25 MMac',
 'mcnn_params': '637.26 k',
 'dcl_macs': '241.26 MMac',
 'dcl_params': '744.65 k',
 'dcl_attn_macs': '74.97 MMac',
 'dcl_attn_params': '868.68 k',
 'DeepSense_macs': '45.11 MMac',
 'DeepSense_params': '325.26 k',
 'GlobalFusion_macs': '38.76 MMac',
 'GlobalFusion_params': '333.64 k',
 'Attend_macs': '226.53 MMac',
 'Attend_params': '667.34 k',
 'ALAE_macs': '190.41 MMac',
 'ALAE_params': '413.13 k',
 'IfConv_macs': '87.39 MMac',
 'IfConv_params': '170.74 k',
 'tinyhar_macs': '12.95 MMac',
 'tinyhar_params': '90.09 k',
 'Vision_MIXER_macs': '213.09 MMac',
 'Vision_MIXER_params': '835.41 k'}

In [7]:
results["dg"]

{'mcnn_macs': '12.72 MMac',
 'mcnn_params': '300.1 k',
 'dcl_macs': '50.69 MMac',
 'dcl_params': '448.45 k',
 'dcl_attn_macs': '16.92 MMac',
 'dcl_attn_params': '572.48 k',
 'DeepSense_macs': '17.62 MMac',
 'DeepSense_params': '323.65 k',
 'GlobalFusion_macs': '9.49 MMac',
 'GlobalFusion_params': '307.52 k',
 'Attend_macs': '52.97 MMac',
 'Attend_params': '444.87 k',
 'ALAE_macs': '49.38 MMac',
 'ALAE_params': '265.03 k',
 'IfConv_macs': '24.46 MMac',
 'IfConv_params': '144.39 k',
 'tinyhar_macs': '2.86 MMac',
 'tinyhar_params': '71.01 k',
 'Vision_MIXER_macs': '54.58 MMac',
 'Vision_MIXER_params': '709.0 k'}

In [8]:
results["dsads"]

{'mcnn_macs': '143.04 MMac',
 'mcnn_params': '859.35 k',
 'dcl_macs': '576.15 MMac',
 'dcl_params': '1.63 M',
 'dcl_attn_macs': '177.01 MMac',
 'dcl_attn_params': '1.75 M',
 'DeepSense_macs': '106.97 MMac',
 'DeepSense_params': '623.51 k',
 'GlobalFusion_macs': '108.61 MMac',
 'GlobalFusion_params': '431.89 k',
 'Attend_macs': '543.88 MMac',
 'Attend_params': '1.33 M',
 'ALAE_macs': '454.44 MMac',
 'ALAE_params': '855.96 k',
 'IfConv_macs': '205.82 MMac',
 'IfConv_params': '248.29 k',
 'tinyhar_macs': '32.68 MMac',
 'tinyhar_params': '145.84 k',
 'Vision_MIXER_macs': '520.22 MMac',
 'Vision_MIXER_params': '1.08 M'}

In [9]:
results["pamap2"]

{'mcnn_macs': '58.25 MMac',
 'mcnn_params': '638.03 k',
 'dcl_macs': '241.26 MMac',
 'dcl_params': '745.43 k',
 'dcl_attn_macs': '74.97 MMac',
 'dcl_attn_params': '869.46 k',
 'DeepSense_macs': '45.11 MMac',
 'DeepSense_params': '325.65 k',
 'GlobalFusion_macs': '35.17 MMac',
 'GlobalFusion_params': '336.72 k',
 'Attend_macs': '226.54 MMac',
 'Attend_params': '668.12 k',
 'ALAE_macs': '190.41 MMac',
 'ALAE_params': '413.52 k',
 'IfConv_macs': '87.4 MMac',
 'IfConv_params': '171.13 k',
 'tinyhar_macs': '12.95 MMac',
 'tinyhar_params': '90.48 k',
 'Vision_MIXER_macs': '213.1 MMac',
 'Vision_MIXER_params': '836.96 k'}

In [6]:
df_re = {}
for key1 in results.keys():
    temp = results[key1]
    df_re["{}_macs".format(key1)] = []
    df_re["{}_params".format(key1)] = []
    for key2 in temp.keys():
        if "macs" in key2:
            value = float(temp[key2].split(" ")[0])
            scale = temp[key2].split(" ")[1]
            if "KM" in scale:
                value = value *1000
            elif "MM" in scale:
                value = value *1000000
            elif "GM" in scale:
                value = value *1000000000
            else:
                assert 1==0
                
            df_re["{}_macs".format(key1)].append(value/1000000)
        if "params" in key2:
            value = float(temp[key2].split(" ")[0])
            scale = temp[key2].split(" ")[1]
            if "k" in scale:
                value = value *1000
            elif "M" in scale:
                value = value *1000000
            elif "G" in scale:
                value = value *1000000000
            else:
                assert 1==0

            
            df_re["{}_params".format(key1)].append(value/1000)

In [11]:
import pandas as pd
pd.DataFrame(df_re).T.to_csv("result.csv")